### Implementing RAG with AutoGen - Agent and Tool

In [ ]:
__import__('pysqlite3')

import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig

import os

from dotenv import load_dotenv
from vector_db_utils import SimpleDocumentIndexer

load_dotenv(override=True)

True

In [4]:
rag_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name="rag_memory",
        persist_directory=os.getenv("CHROMA_PERSIST_DIRECTORY", "chroma_db"),
        embedding_model=os.getenv("EMBEDDING_MODEL", "text-embedding-3-small"),
        vector_search_config={
            "k": 5,
            "distance_metric": "cosine"
        }
    )
)

rag_memory.clear()

<coroutine object ChromaDBVectorMemory.clear at 0x7f1d3fd884f0>

In [8]:
async def index_documents() -> None:
    indexer = SimpleDocumentIndexer(
        memory=rag_memory,
    )

    sources = [
        "https://raw.githubusercontent.com/microsoft/autogen/refs/heads/main/README.md"
        "https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html",
        "https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html",
        "https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/agents.html"
    ]

    chunks:int = await indexer.index_documents(sources)
    
    print(f"Indexed {chunks} chunks from documents and sources into the RAG memory.")

await index_documents()

Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'
Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
/home/vscode/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:10<00:00, 7.58MiB/s]
Failed to send telemetry event CollectionAddEvent: module 'chromadb' has no attribute 'get_settings'


Indexed 47 chunks from documents and sources into the RAG memory.


In [9]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [13]:
rag_assistant = AssistantAgent(
    name="RAGAssistant",
    model_client=model_client,
    system_message="You are a helpful assistant that can answer questions based on the provided documents.",
    description="An assistant that uses RAG to answer questions based on indexed documents.",
    memory=[rag_memory],
)

stream = rag_assistant.run_stream(task="What is AgentChat in AutoGen?")

await Console(stream)

---------- TextMessage (user) ----------
What is AgentChat in AutoGen?


Failed to send telemetry event CollectionQueryEvent: module 'chromadb' has no attribute 'get_settings'


---------- MemoryQueryEvent (RAGAssistant) ----------
[MemoryContent(content='arch Literature Review API Reference PyPi Source AgentChat Agents Agents # AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages. All agents share the following attributes and methods: name : The unique name of the agent. description : The description of the agent in text. run : The method that runs the agent given a task as a string or a list of messages, and returns a TaskResult . Agents are expected to be stateful and this method is expected to be called with new messages, not complete history . run_stream : Same as run() but returns an iterator of messages that subclass BaseAgentEvent or BaseChatMessage followed by a TaskResult as the last item. See autogen_agentchat.messages for more information on AgentChat message types. Assistant Agent # AssistantAgent is a built-in agent that uses a language model and has the ability to use tools. Warning As

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 6, 18, 21, 55, 163396, tzinfo=datetime.timezone.utc), content='What is AgentChat in AutoGen?', type='TextMessage'), MemoryQueryEvent(source='RAGAssistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 6, 18, 21, 55, 717537, tzinfo=datetime.timezone.utc), content=[MemoryContent(content='arch Literature Review API Reference PyPi Source AgentChat Agents Agents # AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages. All agents share the following attributes and methods: name : The unique name of the agent. description : The description of the agent in text. run : The method that runs the agent given a task as a string or a list of messages, and returns a TaskResult . Agents are expected to be stateful and this method is expected to be called with new messages, not complete history . run_st

In [14]:
await rag_memory.close()